# Basic RAG (Retrieval Augmented Generation)

In [1]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
    --------------------------------------- 0.3/13.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/13.7 MB 5.5 MB/s eta 0:00:03
   ------- -------------------------------- 2.6/13.7 MB 5.4 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/13.7 MB 5.4 MB/s eta 0:00:02
   -------------- ------------------------- 5.0/13.7 MB 5.5 MB/s eta 0:00:02
   ----------------- ---------------------- 6.0/13.7 MB 5.5 MB/s eta 0:00:02
   ---------------------- ----------------- 7.6/13.7 MB 5.7 MB/s eta 0:00:02
   -------------------------- ------------- 8.9/13.7 MB 5.8 MB/s eta 0:00:01
   ----------------------------- ---------- 10.2/13.7 MB 5.7 MB/s eta 0:00:01
   --------------------------------- ------ 11.5/13.7 MB 5.8 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.7 MB 5.9 MB/s eta 0:00:01
   ------

In [ ]:
!pip install mistralai

Defaulting to user installation because normal site-packages is not writeable


## Load API key

In [ ]:
import os
os.environ["MISTRAL_API_KEY"] = "YOUR_MISTRAL_API_KEY"
print(f"MISTRAL_API_KEY: {os.environ.get('MISTRAL_API_KEY')}")

MISTRAL_API_KEY: GsUGXiJDxuga92tR85lbI5tDW6U7vnjd


In [4]:
api_key = os.getenv("MISTRAL_API_KEY")

## Get data

- Search for any article and copy its URL.

## Parse the article with BeautifulSoup 
+ +save the text into a text file

In [7]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import requests
from bs4 import BeautifulSoup
import re
response = requests.get("https://www.udst.edu.qa/about-udst/institutional-excellence-ie/policies-and-procedures/sport-and-wellness-facilities-and")
html_doc = response.text
soup = BeautifulSoup(html_doc, "html.parser")
tag = soup.find("div")
text = tag.text
print(text)


















Search













Search by Keyword


Search








Search by Purpose

I am
Select an OptionUDST EmployeeCareer at UDSTParentCurrent StudentAlumniFuture Student



Looking for
Select an Option

Take me There



































Quick links



Current Student



Registration




Academic Calendar




Student Systems




Attendance




Final Exams




Academic Awards




Request Letters and Records




Graduation Ceremony




Graduation Requirements




Ramadan Schedule







Professional Education



Customized Training Courses




Consulting Services




Register Today




Online Resources







Alumni



Presidents Message




Alumni Services




Update Contact Information




Request Letters, Records and Transcripts







Alumni Benefits




UDST Offers







Alumni Events




Alumni Magazine




Alumni Office




Graduation Ceremonies







Testing Centre



Placement Tests




IELTS Tests




Proctoring







Careers




Sport and Wellness



In [11]:
file_name = "Text.txt"
with open(file_name, 'w') as file:
    file.write(text)

## Chunking

In [12]:
chunk_size = 512
chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

In [13]:
len(chunks)

62

## Get embeddings of the chunks

In [14]:
import os
from mistralai import Mistral
def get_text_embedding(list_txt_chunks):
    client = Mistral(api_key=api_key)
    embeddings_batch_response = client.embeddings.create(model="mistral-embed",
                                                         inputs=list_txt_chunks)
    return embeddings_batch_response.data

In [15]:
import numpy as np
text_embeddings = get_text_embedding(chunks)

In [16]:
len(text_embeddings)

62

In [17]:
len(text_embeddings[0].embedding)

1024

In [18]:
embeddings = np.array([text_embeddings[i].embedding for i in range(len(text_embeddings))])

## Store in a vector databsae

In [19]:
import faiss
d = len(text_embeddings[0].embedding)
index = faiss.IndexFlatL2(d)
index.add(embeddings)

## Embed the user query

In [20]:
question = "What are the way I can use sports facilities?"
question_embeddings = np.array([get_text_embedding([question])[0].embedding])

In [21]:
question_embeddings

array([[ 0.03643799,  0.04135132,  0.03222656, ..., -0.01273346,
         0.05230713, -0.00886536]], shape=(1, 1024))

## Search for chunks that are similar to the query

In [22]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[59 56]]


In [23]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['         \n\n                Service registration fees may apply.\n            \n\n\n\n                S&W Facility Reservation Capabilities\n            \n\n                Able to register for various S&W Services\n            \n\n                First Come, First Serve Basis.\n            \n\nSport and Wellness Extended Community Access Hours\nAD HOC Tier\nCollege Partners\n\n\n                Dependent on the nature of facilities and/or services requested, as per the written agreement concluded for this purpose.\n           ', " Staff and Immediate Family of Academic Members and College's administrative staff.\n\n\n                Standard Facility Reservation Rates Apply.\n            \n\n                Service registration fees may apply, in which case rates will be discounted.\n            \n\n\n\n                S&W Facility Reservation Capabilities\n            \n\n                Able to register for various S&W Services\n            \n\n                First Come, First S

In [24]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [25]:
from mistralai import Mistral, UserMessage
def mistral(user_message, model="mistral-small-latest", is_json=False):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)
    messages = [
        UserMessage(content=user_message),
        ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages,
        )
    return chat_response.choices[0].message.content

In [27]:
response = mistral(prompt)
print(response)

Based on the provided context, here are the ways you can use sports facilities:

1. **Service Registration**: You can register for various Sport and Wellness (S&W) services. This is done on a first come, first serve basis.

2. **Ad Hoc Tier and College Partners**: If you are part of the college's ad hoc tier or a college partner, you may have access to the facilities depending on the nature of the facilities and/or services requested, as per a written agreement.

3. **Staff and Family of Academic Members**: If you are a staff member or an immediate family member of academic members or the college's administrative staff, you can use the facilities. Standard facility reservation rates and service registration fees may apply, but rates will be discounted.

4. **Extended Community Access Hours**: You can use the facilities during the extended community access hours.


In [30]:
prompt="What are the main 10 policies that would be taken into consideration when using sport and wellness facilities in udst? (include the policies numbers)"

In [31]:
response = mistral(prompt)
print(response)

When using sport and wellness facilities, especially in a setting like the University of Dubai for Science and Technology (UDST), several key policies would be taken into consideration to ensure safety, fairness, and efficient operation. While specific policy numbers may vary, here are the main policies that would typically be considered:

1. **Health and Safety Policy (Policy No. 001)**
   - Ensures the physical safety of all users through proper equipment maintenance, emergency procedures, and first aid availability.

2. **Access and Eligibility Policy (Policy No. 002)**
   - Defines who is eligible to use the facilities, such as students, faculty, staff, and possibly community members, and specifies the procedures for access.

3. **Facility Usage and Booking Policy (Policy No. 003)**
   - Outlines the procedures for booking facilities, including courts, fields, and gym spaces, to ensure fair and efficient usage.

4. **Equipment Use Policy (Policy No. 004)**
   - Provides guidelines 